In [2]:
import scanpy as sc
import pandas as pd
import anndata as ad
data_path = './data/Slide-seq_Mouse_Cerebellum/'
count_file = 'MappedDGEForR.csv'
meta_file = 'BeadLocationsForR.csv'
# 用pandas读取csv
count0 = pd.read_csv(f'{data_path}/{count_file}')
meta = pd.read_csv(f'{data_path}/{meta_file}')

In [3]:
count0

,Row,CGTAGGGGGAGCCG,GGTTCCAATTTCAC,GTCGGTCTGGGAGG,AAAAAAAAAAAAAA,CCAGTGGTTTTTTT,CGACTACTTTTTTT,GTACAAATTTTTTT,ACACCATACCCCCC,CTCAACTACCCCCC,...,GTATGATTTATCGC,AGCGAGATTTATTA,GAATCGATGCGCTA,GAGGACTTGCGCGC,GCCAAACTCGCGCG,GTTCACGTCGCGAN,GTGCTCGTCGCGCG,GAGAGCCTCGCGTA,CTAAAAGTCGCGCG,CTTACGATATATAG
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610007P14Rik,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610009B22Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0610009E02Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0610009L18Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19125,n-R5s33,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19126,n-R5s45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19127,n-R5s54,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19128,n-R5s93,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#行列转置
count=count0.values[:,1:].T

In [4]:
#提取cellID
cell_id_array = count0.columns[1:]
cell_id_array

Index(['CGTAGGGGGAGCCG', 'GGTTCCAATTTCAC', 'GTCGGTCTGGGAGG', 'AAAAAAAAAAAAAA',
       'CCAGTGGTTTTTTT', 'CGACTACTTTTTTT', 'GTACAAATTTTTTT', 'ACACCATACCCCCC',
       'CTCAACTACCCCCC', 'CTGGCTTAGGGGGG',
       ...
       'GTATGATTTATCGC', 'AGCGAGATTTATTA', 'GAATCGATGCGCTA', 'GAGGACTTGCGCGC',
       'GCCAAACTCGCGCG', 'GTTCACGTCGCGAN', 'GTGCTCGTCGCGCG', 'GAGAGCCTCGCGTA',
       'CTAAAAGTCGCGCG', 'CTTACGATATATAG'],
      dtype='object', length=29275)

In [5]:
#提取geneID
gene_id_array = count0['Row']
gene_id_array

0        0610005C13Rik
1        0610007P14Rik
2        0610009B22Rik
3        0610009E02Rik
4        0610009L18Rik
             ...      
19125          n-R5s33
19126          n-R5s45
19127          n-R5s54
19128          n-R5s93
19129          n-R5s98
Name: Row, Length: 19130, dtype: object

In [7]:
# 提取count_matrix
count_X = count.astype('float')

In [8]:
# 构建adata
adata = ad.AnnData(count_X)
adata.var_names = gene_id_array
adata.obs_names = cell_id_array

/tmp/ipykernel_78446/3501277586.py:2: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(count_X)


In [9]:
meta = pd.read_csv(f'{data_path}/{meta_file}')
meta

,barcodes,xcoord,ycoord
0,CGTAGGGGGAGCCG,1705.258427,3916.898876
1,GGTTCCAATTTCAC,1627.118812,3745.940594
2,GTCGGTCTGGGAGG,1032.081395,2442.790698
3,AAAAAAAAAAAAAA,3688.486486,5990.315315
4,CCAGTGGTTTTTTT,3727.415094,5245.886792
...,...,...,...
29270,GTTCACGTCGCGAN,1059.189189,1525.945946
29271,GTGCTCGTCGCGCG,2732.390533,3646.325444
29272,GAGAGCCTCGCGTA,1958.486486,2828.774775
29273,CTAAAAGTCGCGCG,1253.877358,1249.839623


In [10]:
# 需要把细胞名作为index，与adata的细胞名align起来
meta = meta.set_index('barcodes')

In [11]:
adata.obsm['spatial'] = meta.loc[adata.obs_names].loc[:,['xcoord','ycoord']]

In [12]:
adata

AnnData object with n_obs × n_vars = 29275 × 19130
    obsm: 'spatial'

In [ ]:
adata.write_h5ad(f'{data_path}/puck_180819_11.h5ad')